**Name:** Mylla Pereira de Castro

**Student number:** T00244395

# Genetic Algorithm Assignment
30% of the overall grade for this module

Marks indciated in sections below are based on percentage of marks allocated for this module

In this assignment you must choose a problem, and attempt to use the Genetic Alogrithm that we developed in class to solve this problem.





In [1]:
"""
    Loding required imports and dataset, which is available on this repository
"""

import pandas as pd
from copy import deepcopy
import numpy as np
import random

df = pd.read_csv('grocery_sample.csv')
df.shape

# Filter columns to collect name and number of calories
grocery = df[['item_name','nf_calories']]

## The Problem         **(~30%)**

When we receive our calories consumption requirements from a doctor, it can be extremely tedious to try to meet those needs.

This project is an attempt to, given our calories requirements, predict which food items should be part of a balanced diet.


### Dataset

The dataset used for the project is part of an assessment from the U.S. Department Of Agriculture from 2022. This information [source](https://fdc.nal.usda.gov/) "can be used by, and has benefits for, a variety of users, including researchers, policy makers, academicians and educators, nutrition and health professionals, product developers, and others".  

The particular table selected for this project provides nutricional information, as well as calories (which we will forcus on) for a variety of food items.




*   Discussion of the suitablity of Genetic Algorithms

The dataset used in this project contains a variety of 100 different food items, from different brands, and with different calories values.

Trying to find which X number of items from this list would result in Y targetted calories consumption requires a lot of computer processing if the programmatic approach is linear and iterates to all items.

By making use of a Generic Algorithm, the approach will not only result in a more interesting variety of food items, but it can also provide multiple optimal solucations (different sets of good items every time the code is executed). 


# The problem and the cost function   **(~20%)**

In [2]:
def cost_function(target_calories, individual):
  '''
    Calculates the cost of an individual as the numerical distance from a given target
  '''
  total_calories = 0
  for calories in individual['nf_calories']:
    total_calories += calories
  return abs(target_calories - total_calories)


class Problem:
  '''
    Definition of a problem
    
    Attributes
    ----------

    cost: cost_function
        defines the function to calculate the cost of an individual
    n_items: int
        number of grocery items for an individual
    n_items: int
        number of required calories for an individual
    groceries: obj
        Pandas object with filtering for groceries items names and its number of calories

  '''
  def __init__(self):
    self.cost = cost_function
    self.n_items = 10
    self.target_calories = 2854

    # Filter columns to collect name and number of calories
    self.groceries = df[['item_name','nf_calories']]

# The Individual **(~30%)**


*   Chromosone
*   Crossover
*   Mutation



In [3]:
class Individual:
  '''
    Definition of an individual
    
    Attributes
    ----------

    grocery_items: obj
        Pandas object with a sample of grocery items. The number of items depends on the problem definition
    problem: obj
        an instance of a pre-defined problem
    cost: int
        number of required calories for an individual
    groceries: obj
        Pandas object with all groceries from dataset

  '''

  def __init__(self, problem, groceries):
    self.grocery_items = groceries.sample(n=problem.n_items)
    self.problem = problem
    self.cost = problem.cost(problem.target_calories, self.grocery_items)
    self.groceries = groceries
  

  def crossover(self, other_parent):
    alpha = np.random.randint(len(self.grocery_items))
    child1 = deepcopy(self)
    child2 = deepcopy(other_parent)

    # Get part of the grocery list items from each parent
    child1.grocery_items = pd.concat([self.grocery_items.tail(alpha), other_parent.grocery_items.head(len(self.grocery_items) - alpha)])
    child2.grocery_items = pd.concat([self.grocery_items.tail(len(self.grocery_items) - alpha), other_parent.grocery_items.head(alpha)])
    
    return child1,child2

  def mutate(self):
    # Randomly drops an items from the grocery_items and replace it with a random item from the larger grocery's dataset
    random_index = random.choice(self.grocery_items.index.values)
    mutated_df = self.grocery_items.drop([random_index])
    self.grocery_items = pd.concat([self.groceries.sample(), mutated_df])

## Discussion and justification on the approaches taken for the above

---

A chromosome in this project refers to a dataframe of items that are closer to an ideal target in terms of number of calories. The intention here is to achieve an optimal goal given a list of grocery items.

The crossover function ensures that two children objects are created from two parents, each containing a list of grocery items. Each child will have 50% of the genes of its parents, improving the variety of possible grocery items and as a result, increasing the chances of finding the best solution for the defined problem.

The mutation function will introduce an element of variety to the individual's grocery items by randomly removing an existing indexed row from the list and inserting a random grocery row that can be part of the groceries outside of the individual's grocery list. 

---



## Running the algorithm  **(~10%)**

*   Parameter choices
*   Modifications (if any) to run_genetic
*   Rationale for the above



In [4]:
#  Parameter class here

class Parameters:
  '''
    Definition of parameters
    
    Attributes
    ----------

    number_in_population: int
        number of individuals in population
    maximum_number_of_generations: int
        maximum numer of generations
    child_rate: int
        rate of new children per generation 

  '''
  def __init__(self):
    # Number of individuals in population
    self.number_in_population = 500
    # Maximum numer of generations
    self.maximum_number_of_generations = 50
    self.child_rate = 0.5

In [5]:
def choose_two_indices_from(max_number):
  ind1 = np.random.randint(max_number)
  ind2 = np.random.randint(max_number)
  if ind1 == ind2:
    return choose_two_indices_from(max_number)

  return ind1,ind2

# Run Genetic method here
def run_genetic(problem:Problem, params:Parameters):
  # read the problem
  cost_function = problem.cost

  # read parameters
  current_population_size = params.number_in_population
  num_children = params.child_rate * current_population_size

  # setting best solution
  best_solution = None

  # initialise population
  population = []
  for _ in range(current_population_size):
    i = Individual(problem, problem.groceries)
    population.append(i)
    if best_solution is None or i.cost < best_solution.cost:
      best_solution = deepcopy(i)

  # Generation Loop:
  for generation in range(1, params.maximum_number_of_generations):
    # generate children
    children = []
    while len(children) < num_children:
      parent1_index, parent2_index = choose_two_indices_from(current_population_size)
      parent1 = population[parent1_index]
      parent2 = population[parent2_index]

      child1, child2 = parent1.crossover(parent2)
      child1.mutate()
      child2.mutate()

      #cost of children
      child1.cost = cost_function(problem.target_calories, child1.grocery_items)
      child2.cost = cost_function(problem.target_calories, child2.grocery_items)


      # add to children list
      children.append(child1)
      children.append(child2)
    
    # add children to population
    population += children

    # sort the population
    population = sorted(population,key= lambda x: x.cost)

    # cull population
    population = population[:current_population_size]

    # retrieve best solution
    if population[0].cost < best_solution.cost:
      best_solution = deepcopy(population[0])
      print(f"Generation {generation} has cost of {best_solution.cost}")
  

  return population, best_solution

In [6]:
#  Running of the algorithm with outputs here
population, best_solution = run_genetic(Problem(), Parameters())
print(f"Best cost of {best_solution.problem.n_items} grocery items is {best_solution.cost}. Grocery items selection: \n")
best_solution.grocery_items

Generation 4 has cost of 424
Generation 5 has cost of 289
Generation 7 has cost of 279
Generation 8 has cost of 106
Generation 11 has cost of 99
Generation 12 has cost of 19
Generation 14 has cost of 16
Generation 15 has cost of 8
Generation 17 has cost of 2
Generation 25 has cost of 1
Best cost of 10 grocery items is 1. Grocery items selection: 



,item_name,nf_calories
21,"Warm Delights, Molten Chocolate Cake",370
21,"Warm Delights, Molten Chocolate Cake",370
88,Meatballs,270
68,Santa Fe Style Salad,280
55,Fudge Double Filled Twist & Shout Sandwich Coo...,140
6,"Maintenance Elemental Diet, Unflavored Powder",375
59,"Mini Cakes, Creme Filled Chocolate, Cakes, Ind...",220
68,Santa Fe Style Salad,280
47,"Milk, 2% Reduced Fat",230
92,Grilled Asiago Chicken & Penne Pasta,320


In [8]:
#  If changes to params or reruns of iterations dont overwrite, create more cells and copy code down to show evolution of final solution

## Second execution of the code with different results
population, best_solution = run_genetic(Problem(), Parameters())
print(f"Best cost of {best_solution.problem.n_items} items is {best_solution.cost}. Grocery items selection: \n")
best_solution.grocery_items

Generation 1 has cost of 628
Generation 4 has cost of 474
Generation 6 has cost of 314
Generation 8 has cost of 254
Generation 9 has cost of 205
Generation 10 has cost of 64
Generation 13 has cost of 1
Best cost of 10 items is 1. Grocery items selection: 



,item_name,nf_calories
44,"Sausage, Smoked",280
48,Beef Burgers,290
6,"Maintenance Elemental Diet, Unflavored Powder",375
4,"Vegi-Dressing, Creamy Dill",70
48,Beef Burgers,290
35,"Ice Cream Bar, Double Caramel",340
99,Bean and Beef Enchiladas,210
7,Beef Burgers,290
80,"Ice Cream Bar, Double Caramel",340
21,"Warm Delights, Molten Chocolate Cake",370


## Results and conclusions    **(~10%)**

---

A variety of features could be implemented to cover different use cases eg. gathering grocery items based on a list of X nutrients required for minimum daily consumption. This would increase the complexity of the issue with interesting results.

My first impression of a Genetic Algorithm was that it was quite a limited algorithm that was probably used in specific use cases. However, by doing a quick search I was able to realise that this approach to find the best solution to a problem that has no clear answers has a number of real world applications in areas such as robotics, medicine, marketing and for scheduling tasks.

In this project, I was able to see how the change on parameters such as the size of a population and number of generations can impact on the best found solution. Not only that but also how the size of the dataset used or created can impact the performance of the execution even with constantly improved libraries such as Numpy.

It was also interisting to correlate the knowledge acquired in the Big Data model and implement these skills to treat and read the data for this project.

---

